In [1]:
import numpy as np
import pandas as pd
from surprise import Dataset
from lib.BooleanMatrixBasedRecomenders import FcaBmf, cosine_distance

from rich.jupyter import print

In [2]:
# Create a trainset from the complemte MovieLens 100K dataset
dataset = Dataset.load_builtin("ml-100k")
trainset = dataset.build_full_trainset()

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/brodrigues/.surprise_data/ml-100k


In [ ]:
results = []

for coverage in [1.0, 0.8, 0.6]:
    algo = FcaBmf(coverage=coverage, distance_strategy=cosine_distance, verbose=True)
    algo.fit(trainset)

    result = []

    result = algo.actual_coverage)
    result = algo.number_of_factors)
    results.append(result)



In [4]:
pd.DataFrame(results, columns=['Actual Coverage', "Number of factors"], index=["100%", "80%", "60%"])

,Actual Coverage,Number of factors
100%,NaN,NaN
80%,NaN,NaN
60%,NaN,NaN


In [3]:
import itertools
from surprise.model_selection import KFold

kf = KFold(n_splits=5)
# trainset, testset = next(kf.split(dataset))

data = []
for trainset, testset in kf.split(dataset):
    data.append((trainset, testset))

coverages = [1.0, 0.8, 0.6]
ks = [1, 5, 10, 20, 30, 40 ,50 , 60]

thread_args = [d for d in itertools.product(coverages, ks, data)]

In [10]:
kf = KFold(n_splits=5)
trainset, testset = next(kf.split(dataset))

algo = FcaBmf(coverage=1.0, distance_strategy=cosine_distance, k=30)
algo.fit(trainset)
predictions = algo.test(testset)


In [54]:
import statistics
from collections import defaultdict

def get_global_precision(predictions, relevance_threshold = 1):
    """
    Returns the global precision, or micro-averaged precision, from a predictions list.

    Precision is defined as the fraction of relevant instances among the retrieved instances. In recommender systems,
    it measures the fraction of items that are liked by the user among the items that are recommended by the system.
    
    Precision = True Positives / (True Positives + False Positives)

    For example, if you have a recommender system that suggests movies to a user, and you have 100 movies in total,
    10 of which are liked by the user. If your system recommends 8 movies to the user, 4 of which are liked by the user
    (true positives), but also 4 of which are disliked by the user (false positives), then your precision is 4 / (4 + 4)
    = 0.5. This means that 50% of the movies that your system recommended were actually liked by the user.
    
    Global precision gives equal weight to each item, regardless of which user rated or was recommended it.
    """

    def is_relevant(measure):
        return measure >= relevance_threshold

    true_positives = 0
    false_positives = 0

    for _, _, true_rating, estimate, _ in predictions:

        if is_relevant(estimate):
            if is_relevant(true_rating):
                true_positives += 1
            else:
                false_positives += 1

    return true_positives / (true_positives + false_positives)


def get_user_averaged_precision(predictions, relevance_threshold = 1):

    def is_relevant(measure):
        return measure >= relevance_threshold

    precisions = []
    ratings_per_user = defaultdict(list)
    for user_id, _, true_rating, estimate, _ in predictions:
        ratings_per_user[user_id].append((estimate, true_rating))

    for _, user_ratings in ratings_per_user.items():
        
        true_positives = 0
        false_positives = 0

        for (estimate, true_rating) in user_ratings:
        
            if is_relevant(estimate):
                if is_relevant(true_rating):
                    true_positives += 1
                else:
                    false_positives += 1

        try:
            precision = true_positives / (true_positives + false_positives)
        except ZeroDivisionError:
            pass
        else:
            precisions.append(precision)

    return statistics.mean(precisions)


def get_precision_at_k(predictions, relevance_threshold = 1, k = 20):

    def is_relevant(measure):
        return measure >= relevance_threshold

    precisions = []
    ratings_per_user = defaultdict(list)
    for user_id, _, true_rating, estimate, _ in predictions:
        ratings_per_user[user_id].append((estimate, true_rating))

    for _, user_ratings in ratings_per_user.items():
        
        relevant_itens_in_the_top_k = 0

        user_ratings.sort(key=lambda x: x[0], reverse=True)

        for (estimate, true_rating) in user_ratings[:k]:
            if is_relevant(true_rating):
                relevant_itens_in_the_top_k += 1

        precisions.append(relevant_itens_in_the_top_k/k)
            
    return statistics.mean(precisions)


def get_global_recall(predictions, relevance_threshold = 1):
    """
    Returns the recall from a predictions list.

    Recall is defined as the fraction of relevant instances that were retrieved. In recommender systems,
    it measures the fraction of items that are liked by the user among all the items that are available.
    
    Recall = True Positives / (True Positives + False Negatives)

    For example, if you have a recommender system that suggests movies to a user, and you have 100 movies in total,
    10 of which are liked by the user. If your system misses 6 movies that are liked by the user (false negatives), then
    your recall is 4 / (4 + 6) = 0.4. This means that 40% of the movies that are actually liked by the user were
    recommended by your system.
    
    """

    def is_relevant(measure):
        return measure >= relevance_threshold

    true_positives = 0
    false_negatives = 0

    for _, _, true_rating, estimate, _ in predictions:

        if is_relevant(estimate):
            if is_relevant(true_rating):
                true_positives += 1
        else:
            if is_relevant(true_rating):
                false_negatives += 1

    return true_positives / (true_positives + false_negatives)


def get_user_averaged_recall(predictions, relevance_threshold = 1):

    def is_relevant(measure):
        return measure >= relevance_threshold

    recalls = []
    ratings_per_user = defaultdict(list)
    for user_id, _, true_rating, estimate, _ in predictions:
        ratings_per_user[user_id].append((estimate, true_rating))

    for _, user_ratings in ratings_per_user.items():
        
        true_positives = 0
        false_negatives = 0

        for (estimate, true_rating) in user_ratings:
        
            if is_relevant(estimate):
                if is_relevant(true_rating):
                    true_positives += 1
            else:
                if is_relevant(true_rating):
                    false_negatives += 1
        try:
            recall = true_positives / (true_positives + false_negatives)
        except ZeroDivisionError:
            pass
        else:
            recalls.append(recall)

    return statistics.mean(recalls)


def get_recall_at_k(predictions, relevance_threshold = 1, k = 20):

    def is_relevant(measure):
        return measure >= relevance_threshold

    recalls = []
    ratings_per_user = defaultdict(list)
    for user_id, _, true_rating, estimate, _ in predictions:
        ratings_per_user[user_id].append((estimate, true_rating))

    for _, user_ratings in ratings_per_user.items():
        
        relevant_itens_in_the_top_k = 0
        total_relevant_itens = 0

        user_ratings.sort(key=lambda x: x[0], reverse=True)

        for (estimate, true_rating) in user_ratings[:k]:
            if is_relevant(true_rating):
                relevant_itens_in_the_top_k += 1

        for (estimate, true_rating) in user_ratings:
            if is_relevant(true_rating):
                total_relevant_itens += 1
        try:
            recalls.append(relevant_itens_in_the_top_k/total_relevant_itens)
        except ZeroDivisionError:
            pass
            
    return statistics.mean(recalls)


# print(get_precision_at_k(predictions = predictions, relevance_threshold = 5, k = 20))
# print(get_user_averaged_precision(predictions=predictions, relevance_threshold=3))
# print(get_global_precision(predictions=predictions, relevance_threshold=3))
# print(get_global_recall(predictions=predictions, relevance_threshold=3))


In [14]:
from surprise.accuracy import mae, rmse

def work(coverage, k, dataset_tuple):

    result = {}

    (trainset, testset) = dataset_tuple
    
    algo = FcaBmf(coverage=coverage, distance_strategy=cosine_distance, k=k)
    algo.fit(trainset)
    predictions = algo.test(testset)

    result['predictions'] = predictions

    result['actual_coverage'] = algo.actual_coverage
    result['number_of_factors'] = algo.number_of_factors

    return coverage, k, result

In [15]:
from multiprocessing import Pool

with Pool(16) as pool:
    raw_results = pool.starmap(work, iterable=thread_args)


/usr/local/lib/python3.8/dist-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.8/dist-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.8/dist-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.8/dist-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.8/dist-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.8/dist-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv

In [56]:
RELEVANCE_THRESHOLD = 5
NUMBER_OF_TOP_RECOMMENDATIONS = 20

for (coverage, k, result) in raw_results:
    result['mae'] = mae(predictions = result['predictions'], verbose=False)
    result['rmse'] = rmse(predictions = result['predictions'], verbose=False)

    result['global_recall'] = get_global_recall(predictions = result['predictions'], relevance_threshold = RELEVANCE_THRESHOLD)
    result['user_averaged_recall'] = get_user_averaged_recall(predictions = result['predictions'], relevance_threshold = RELEVANCE_THRESHOLD)
    result['recall_at_k'] = get_recall_at_k(predictions = result['predictions'], relevance_threshold = RELEVANCE_THRESHOLD, k = NUMBER_OF_TOP_RECOMMENDATIONS)

    result['global_precision'] = get_global_precision(predictions = result['predictions'], relevance_threshold = RELEVANCE_THRESHOLD)
    result['user_averaged_precision'] = get_user_averaged_precision(predictions = result['predictions'], relevance_threshold = RELEVANCE_THRESHOLD)
    result['precision_at_k'] = get_precision_at_k(predictions = result['predictions'], relevance_threshold = RELEVANCE_THRESHOLD, k = NUMBER_OF_TOP_RECOMMENDATIONS)

In [57]:
results = {}

for coverage in coverages:
    results[coverage] = {}
    for k in ks:
        results[coverage][k] = defaultdict(list)

for (coverage, k, result) in raw_results:
    # print(coverage, k, result['number_of_factors'])

    results[coverage][k]['actual_coverages'].append(result['actual_coverage'])
    results[coverage][k]['numbers_of_factors'].append(result['number_of_factors'])

    results[coverage][k]['maes'].append(result['mae'])
    results[coverage][k]['rmses'].append(result['rmse'])

    results[coverage][k]['global_recalls'].append(result['global_recall'])
    results[coverage][k]['user_averaged_recalls'].append(result['user_averaged_recall'])
    results[coverage][k]['recalls_at_k'].append(result['recall_at_k'])

    results[coverage][k]['global_precisions'].append(result['global_precision'])
    results[coverage][k]['user_averaged_precisions'].append(result['user_averaged_precision'])
    results[coverage][k]['precisions_at_k'].append(result['precision_at_k'])

    # results[coverage][k]['f1s'].append(result['f1'])
    # results[coverage][k]['predictions_sets'].append(result['predictions'])


In [98]:
import json

with open('result.json', 'w') as f:
    json.dump(results, f, indent=4)

In [58]:
import statistics

actual_coverages_curves = defaultdict(list)
numbers_of_factors_curves = defaultdict(list)

mae_curves = defaultdict(list)
rmse_curves = defaultdict(list)

global_recall_curves = defaultdict(list)
user_averaged_recall_curves = defaultdict(list)
recalls_at_k_curve = defaultdict(list)

global_precision_curves = defaultdict(list)
user_averaged_precision_curves = defaultdict(list)
precisions_at_k_curve = defaultdict(list)

for coverage in coverages:
    for k in ks:

        actual_coverages_curves[coverage].append(statistics.mean(results[coverage][k]['actual_coverages']))
        numbers_of_factors_curves[coverage].append(statistics.mean(results[coverage][k]['numbers_of_factors']))

        rmse_curves[coverage].append(statistics.mean(results[coverage][k]['rmses']))
        mae_curves[coverage].append(statistics.mean(results[coverage][k]['maes']))

        global_recall_curves[coverage].append(statistics.mean(results[coverage][k]['global_recalls']))
        user_averaged_recall_curves[coverage].append(statistics.mean(results[coverage][k]['user_averaged_recalls']))
        recalls_at_k_curve[coverage].append(statistics.mean(results[coverage][k]['recalls_at_k']))

        global_precision_curves[coverage].append(statistics.mean(results[coverage][k]['global_precisions']))
        user_averaged_precision_curves[coverage].append(statistics.mean(results[coverage][k]['user_averaged_precisions']))
        precisions_at_k_curve[coverage].append(statistics.mean(results[coverage][k]['precisions_at_k']))


In [59]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=5, cols=2, subplot_titles=(
        'Actual Coverage', 'Number of factors', 
        'MAE', 'RMSE',
        'Global Recall', 'User Averaged Recall',
        'Global Precision', 'User Averaged Precision',
        'Recall@K', 'Precision@K',
        ))

fig.update_xaxes(title_text='Number of nearest neighbors used (k)')

fig.update_yaxes(title_text='Actual Coverage', row=1, col=1)
fig.update_yaxes(title_text='Number of factors', row=1, col=2)

fig.update_yaxes(title_text='MAE', row=2, col=1)
fig.update_yaxes(title_text='RMSE', row=2, col=2)

fig.update_yaxes(title_text='Global Recall', row=3, col=1)
fig.update_yaxes(title_text='User Averaged Recall', row=3, col=2)

fig.update_yaxes(title_text='Global Precision', row=4, col=1)
fig.update_yaxes(title_text='User Averaged Precision', row=4, col=2)

fig.update_yaxes(title_text='Recall@K', row=5, col=1)
fig.update_yaxes(title_text='Precision@K', row=5, col=2)

fig.update_layout(legend_title_text='Rating Coverage Level', height=2000)

line_color = {1.0: dict(color='red'), 0.8: dict(color='green'),0.6: dict(color='blue'),}

for coverage in coverages:

    name_string = f"{coverage*100}%"

    fig.add_trace(go.Scatter(x=ks, y=actual_coverages_curves[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=True), row=1, col=1)
    fig.add_trace(go.Scatter(x=ks, y=numbers_of_factors_curves[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=1, col=2)

    fig.add_trace(go.Scatter(x=ks, y=mae_curves[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=ks, y=rmse_curves[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=2, col=2)

    fig.add_trace(go.Scatter(x=ks, y=global_recall_curves[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=3, col=1)
    fig.add_trace(go.Scatter(x=ks, y=user_averaged_recall_curves[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=3, col=2)

    fig.add_trace(go.Scatter(x=ks, y=global_precision_curves[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=4, col=1)
    fig.add_trace(go.Scatter(x=ks, y=user_averaged_precision_curves[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=4, col=2)

    fig.add_trace(go.Scatter(x=ks, y=recalls_at_k_curve[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=5, col=1)
    fig.add_trace(go.Scatter(x=ks, y=precisions_at_k_curve[coverage], mode='lines+markers', name=name_string, line=line_color[coverage], showlegend=False), row=5, col=2)


fig.show()



# results[coverage][k]['actual_coverages'].append(result['actual_coverage'])
# results[coverage][k]['numbers_of_factors'].append(result['number_of_factors'])
# results[coverage][k]['maes'].append(result['mae'])
# results[coverage][k]['rmses'].append(result['rmse'])
# results[coverage][k]['recalls'].append(result['recall'])
# results[coverage][k]['precisions'].append(result['precision'])
# results[coverage][k]['f1s'].append(result['f1'])